In [1]:
import math

import jupyter_cadquery as jcq
from jupyter_cadquery.replay import show

from build123d import *

Overwriting auto display for cadquery Workplane and Shape


In [2]:
jcq.set_defaults(
    control="orbit",
    axes=True,
    axes0=True,
    grid=(True, False, False),
    glass=True,
    theme="browser",
)

In [3]:
%%html
<style>
   /* Fix widget background color in VS Code */
   .cell-output-ipywidget-background {
      background-color: transparent !important;
   }
</style>

In [33]:
panel_w = 469
panel_thickness = 3
screw_depth = 3
screw_r = 3 / 2 + 0.1
screw_head_r = 6 / 2
motor_mount_flange = 6.5
connector_w = 10
connector_thickness = panel_thickness / 2

In [5]:
with BuildPart() as deck_panel:
    with BuildSketch():
        with Locations((-(469 - 444), 41.5 + 3)):
            edges = import_svg("voron_trident_350_deck.svg")
            add(edges)
        f = make_face(mode=Mode.PRIVATE)
        mirror(f, about=Plane.YZ)
    extrude(amount=panel_thickness)
show(deck_panel)

CadViewerWidget(anchor=None, cad_width=800, glass=True, height=600, pinning=False, theme='browser', title=None…

In [8]:
front_motor_hole_w = 41.5 + 3
front_w = front_motor_hole_w + 10
with BuildPart() as panel_front_left:
    split(deck_panel.part, bisect_by=Plane.XZ.offset(-front_w), keep=Keep.TOP)
    split(
        panel_front_left.part, bisect_by=Plane.YZ.offset(panel_w / 2), keep=Keep.BOTTOM
    )
    with Locations((0, front_w)):
        with BuildSketch():
            Rectangle(screw_depth * 2, 10, align=[Align.MIN, Align.MAX])
        extrude(amount=-(20 - panel_thickness))
    with Locations(panel_front_left.faces().filter_by(Axis.X)[1]):
        with Locations((0, panel_thickness / 2, 0)):
            CounterBoreHole(screw_r, screw_head_r, screw_depth)
    with Locations((panel_w / 2 - 50, 0)):
        with BuildSketch():
            Rectangle(10, screw_depth * 2, align=(Align.MAX, Align.MIN))
        m = extrude(amount=-(20 - panel_thickness))
        with Workplanes(m.faces().filter_by(Axis.Y)[1]):
            with Locations((0, -panel_thickness / 2, 0)):
                CounterBoreHole(screw_r, screw_head_r, screw_depth)
    with Locations((front_w, 0)):
        with BuildSketch():
            Rectangle(10, screw_depth * 2, align=(Align.MAX, Align.MIN))
        m = extrude(amount=-(20 - panel_thickness))
        with Workplanes(m.faces().filter_by(Axis.Y)[1]):
            with Locations((0, -panel_thickness / 2, 0)):
                CounterBoreHole(screw_r, screw_head_r, screw_depth)
    with Locations((panel_w / 2, 0)):
        with BuildSketch():
            Rectangle(connector_w / 2, panel_w, align=(Align.MAX, Align.MIN))
        extrude(amount=connector_thickness, mode=Mode.SUBTRACT)

show(panel_front_left.part)

with BuildPart() as panel_front_right:
    mirror(panel_front_left.part, about=Plane.YZ)

with BuildPart() as front_connector:
    Box(front_w, connector_w, connector_thickness)

panel_front_left.part.export_step("/tmp/panel_front_left.step")
panel_front_right.part.export_step("/tmp/panel_front_right.step")
front_connector.part.export_step("/tmp/front_connector.step")

CadViewerWidget(anchor=None, cad_width=800, glass=True, height=600, pinning=False, theme='browser', title=None…


*******************************************************************
******        Statistics on Transfer (Write)                 ******

*******************************************************************
******        Transfer Mode = 0  I.E.  As Is       ******
******        Transferring Shape, ShapeType = 0                      ******
** WorkSession : Sending all data
 Step File Name : /tmp/panel_front_left.step(1974 ents)  Write  Done

*******************************************************************
******        Statistics on Transfer (Write)                 ******

*******************************************************************
******        Transfer Mode = 0  I.E.  As Is       ******
******        Transferring Shape, ShapeType = 0                      ******
** WorkSession : Sending all data
 Step File Name : /tmp/panel_front_right.step(1992 ents)  Write  Done

*******************************************************************
******        Statistics on Transfer (Write)

<IFSelect_ReturnStatus.IFSelect_RetDone: 1>

In [45]:
rear_motor_hole = (52, 43)
rear_motor_ext = (13, 8.5)
rear_w = rear_motor_hole[1] + rear_motor_ext[1] + 10
rear_cable_hole = (14,12)
with BuildPart() as panel_rear_base:
    with Locations((-panel_w / 2, -panel_w + rear_w)):
        r = split(
            deck_panel.part,
            bisect_by=Plane.XZ.offset(-(panel_w - rear_w)),
            keep=Keep.BOTTOM,
            mode=Mode.PRIVATE,
        )
        add(r)
        # split(panel_rare_left.part, bisect_by=Plane.YZ.offset(panel_w/2), keep=Keep.BOTTOM)

with BuildPart() as panel_rear_support_left:
    with Locations((-panel_w/2, 0)):
        with BuildSketch():
            Rectangle(screw_depth * 2, 10, align=[Align.MIN, Align.MIN])
        m = extrude(amount=-(20 - panel_thickness))
        with Workplanes(m.faces().filter_by(Axis.X)[1]):
            with Locations((0, panel_thickness / 2, 0)):
                CounterBoreHole(screw_r, screw_head_r, screw_depth)
    with Locations((-panel_w/2+rear_cable_hole[0], rear_w)):
        with BuildSketch():
            Rectangle(10, screw_depth * 2, align=(Align.MIN, Align.MAX))
        m = extrude(amount=-(20 - panel_thickness))
        with Workplanes(m.faces().filter_by(Axis.Y)[0]):
            with Locations((0, panel_thickness / 2, 0)):
                CounterBoreHole(screw_r, screw_head_r, screw_depth)
    with Locations((-(rear_motor_hole[0]/2+rear_motor_ext[0]), rear_w)):
        with BuildSketch():
            Rectangle(10, screw_depth * 2, align=(Align.MAX, Align.MAX))
        m = extrude(amount=-(20 - panel_thickness))
        with Workplanes(m.faces().filter_by(Axis.Y)[0]):
            with Locations((0, panel_thickness / 2, 0)):
                CounterBoreHole(screw_r, screw_head_r, screw_depth)
 
with BuildPart() as panel_rear:
    add(panel_rear_base.part)
    with Locations((0, 0)):
        with BuildSketch():
            Rectangle(connector_w, panel_w, align=(Align.CENTER, Align.MIN))
        extrude(amount=connector_thickness, mode=Mode.SUBTRACT)
    add(panel_rear_support_left.part)
    mirror(panel_rear_support_left.part, about=Plane.YZ)
    split(panel_rear.part, bisect_by=Plane.YZ, keep=Keep.BOTH)
show(panel_rear)

(panel_rear_left, panel_rear_right) = panel_rear.solids().sort_by(Axis.X)

panel_rear_left.export_step("/tmp/panel_rear_left.step")
panel_rear_right.export_step("/tmp/panel_rear_right.step")

CadViewerWidget(anchor=None, cad_width=800, glass=True, height=600, pinning=False, theme='browser', title=None…


*******************************************************************
******        Statistics on Transfer (Write)                 ******

*******************************************************************
******        Transfer Mode = 0  I.E.  As Is       ******
******        Transferring Shape, ShapeType = 2                      ******
** WorkSession : Sending all data
 Step File Name : /tmp/panel_rear_left.step(2590 ents)  Write  Done

*******************************************************************
******        Statistics on Transfer (Write)                 ******

*******************************************************************
******        Transfer Mode = 0  I.E.  As Is       ******
******        Transferring Shape, ShapeType = 2                      ******
** WorkSession : Sending all data
 Step File Name : /tmp/panel_rear_right.step(2302 ents)  Write  Done


<IFSelect_ReturnStatus.IFSelect_RetDone: 1>

In [42]:
connector_x = 0.8
with BuildPart() as connector_v2:
    with BuildSketch() as sk:
        Rectangle(
            (connector_w + connector_x) / 2,
            connector_thickness * 2 + panel_thickness - connector_thickness,
            align=(Align.MIN, Align.CENTER),
        )
        with Locations((connector_x / 2, 0)):
            Rectangle(
                connector_w / 2,
                panel_thickness - connector_thickness + 0.1,
                align=(Align.MIN, Align.CENTER),
                mode=Mode.SUBTRACT,
            )
        mirror(sk.sketch, Plane.YZ)
    extrude(amount=front_w)
    #with Locations((0, (panel_thickness-connector_thickness+0.1)/2, 0)):
    #    Box(
    #        connector_x + connector_w,
    #        connector_thickness,
    #        5,
    #        align=(Align.CENTER, Align.MIN, Align.MIN),
    #        mode=Mode.SUBTRACT
    #    )
show(connector_v2)
connector_v2.part.export_step("/tmp/connector_v2.step")

CadViewerWidget(anchor=None, cad_width=800, glass=True, height=600, pinning=False, theme='browser', title=None…


*******************************************************************
******        Statistics on Transfer (Write)                 ******

*******************************************************************
******        Transfer Mode = 0  I.E.  As Is       ******
******        Transferring Shape, ShapeType = 0                      ******
** WorkSession : Sending all data
 Step File Name : /tmp/connector_v2.step(1080 ents)  Write  Done


<IFSelect_ReturnStatus.IFSelect_RetDone: 1>